In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)

train_data = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
train_data.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1


In [2]:
test_data = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')
test_data.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, stay safe everyone."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all"
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [3]:
sample_submission = pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')
sample_submission.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [4]:
def describe_data(df):
    
    print('Data types:')
    print(df.dtypes)

    print()
    
    print('Rows and columns:')
    print(df.shape)
    
    print()
    
    print('Column names:')
    print(df.columns)
    
    print()
    
    print('Null values:')
    print(df.apply(lambda x: sum(x.isnull()) / len(df)))

describe_data(train_data)

Data types:
id          int64 
keyword     object
location    object
text        object
target      int64 
dtype: object

Rows and columns:
(7613, 5)

Column names:
Index(['id', 'keyword', 'location', 'text', 'target'], dtype='object')

Null values:
id          0.000000
keyword     0.008013
location    0.332720
text        0.000000
target      0.000000
dtype: float64


In [5]:
train_data = train_data.drop(['id', 'keyword', 'location'], axis=1)
train_data.head()

,text,target
0,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1
3,"13,000 people receive #wildfires evacuation orders in California",1
4,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1


In [6]:
import re

def clean_text(df, text_field):
    df[text_field] = df[text_field].str.lower()
    df[text_field] = df[text_field].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  # nts: unsure exactly what this is doing
    return df

data_clean = clean_text(train_data, "text")
data_clean.head()

,text,target
0,our deeds are the reason of this earthquake may allah forgive us all,1
1,forest fire near la ronge sask canada,1
2,all residents asked to shelter in place are being notified by officers no other evacuation or shelter in place orders are expected,1
3,13000 people receive wildfires evacuation orders in california,1
4,just got sent this photo from ruby alaska as smoke from wildfires pours into a school,1


In [7]:
from nltk.corpus import stopwords

stop = stopwords.words('english')
data_clean['text'] = data_clean['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

data_clean.head()

,text,target
0,deeds reason earthquake may allah forgive us,1
1,forest fire near la ronge sask canada,1
2,residents asked shelter place notified officers evacuation shelter place orders expected,1
3,13000 people receive wildfires evacuation orders california,1
4,got sent photo ruby alaska smoke wildfires pours school,1


In [8]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(data_clean['text'], data_clean['target'], random_state=0) 

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier

pipeline_sgd = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('nb', SGDClassifier()),
])

model = pipeline_sgd.fit(X_train, y_train)

In [10]:
from sklearn.metrics import classification_report

y_preds = model.predict(X_valid)
print(classification_report(y_valid, y_preds))

              precision    recall  f1-score   support

           0       0.80      0.85      0.83      1107
           1       0.78      0.71      0.74       797

    accuracy                           0.79      1904
   macro avg       0.79      0.78      0.78      1904
weighted avg       0.79      0.79      0.79      1904



In [11]:
test_data_clean = test_data.copy()
test_data_clean = clean_text(test_data_clean, "text")
test_data_clean['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
test_data_clean = test_data_clean['text']

test_data_clean.head()

0    just happened a terrible car crash                                                            
1    heard about earthquake is different cities stay safe everyone                                 
2    there is a forest fire at spot pond geese are fleeing across the street i cannot save them all
3    apocalypse lighting spokane wildfires                                                         
4    typhoon soudelor kills 28 in china and taiwan                                                 
Name: text, dtype: object

In [12]:
test_preds = model.predict(test_data_clean)

In [13]:
id_col = test_data['id']
submission_df_1 = pd.DataFrame({
    'id': id_col,
    'target': test_preds
})

submission_df_1.head()

,id,target
0,0,1
1,2,0
2,3,1
3,9,1
4,11,1


In [14]:
submission_df_1.to_csv('submission_1.csv', index=False)